In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin





class absenteeism_model():
      
        def __init__(self, pipe_file):
            # read the 'pipline' file which is saved
            with open('pipeline','rb') as pipe_file:
                self.pipeline = pickle.load(pipe_file)
                self.data = None
        
        # take a data file (*.csv) and preprocess it in the same way as in the lectures
        def load_and_clean_data(self, data_file):
            
            # import the data
            df = pd.read_csv(data_file,delimiter=',')
            # store the data in a new variable for later use
            self.df_with_predictions = df.copy()
            # drop the 'ID' column
            df = df.drop(['ID'], axis = 1)
            # to preserve the code we've created in the previous section, we will add a column with 'NaN' strings
            df['Absenteeism Time in Hours'] = 'NaN'

            # create a separate dataframe, containing dummy values for ALL avaiable reasons
            reason_columns = pd.get_dummies(df['Reason for Absence'], drop_first = True)
            
            # split age_columns into 4 types
            Age1 = pd.get_dummies(df['Age']).loc[:,27:29].max(axis = 1)
            Age2 = pd.get_dummies(df['Age']).loc[:,30:39].max(axis = 1)
            Age3 = pd.get_dummies(df['Age']).loc[:,40:49].max(axis = 1)
            Age4 = pd.get_dummies(df['Age']).loc[:,50:].max(axis = 1)
            
            # to avoid multicollinearity, drop the 'Reason for Absence' column from df
            df = df.drop(['Age'], axis = 1)
            
            
            # concatenate df and the 4 types of reason for absence
            df = pd.concat([df, Age1, Age2, Age3, Age4], axis = 1)
            
            df.reindex(columns = ['Reason for Absence', 'Date', 'Transportation Expense',
           'Distance to Work', 'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children', 
                            'Pets', 'Age1', 'Age2', 'Age3', 'Age4', 'Absenteeism Time in Hours'])
            
            # split reason_columns into 4 types
            reason_type_1 = reason_columns.loc[:,1:14].max(axis=1)
            reason_type_2 = reason_columns.loc[:,15:17].max(axis=1)
            reason_type_3 = reason_columns.loc[:,18:21].max(axis=1)
            reason_type_4 = reason_columns.loc[:,22:].max(axis=1)
            
            # to avoid multicollinearity, drop the 'Reason for Absence' column from df
            df = df.drop(['Reason for Absence'], axis = 1)
            
            # concatenate df and the 4 types of reason for absence
            df = pd.concat([df, reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis = 1)
            
            # assign names to the 4 reason type columns
            # note: there is a more universal version of this code, however the following will best suit our current purposes             
            column_names =  ['Date', 'Transportation Expense',
           'Distance to Work', 'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children', 
                            'Pets', 'Age1', 'Age2', 'Age3', 'Age4', 'Absenteeism Time in Hours','Reason_1', 'Reason_2', 'Reason_3', 'Reason_4']
            #['Date', 'Transportation Expense', 'Distance to Work', 'Age',
             #              'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children',
               #            'Pet', 'Absenteeism Time in Hours', 'Reason_1', 'Reason_2', 'Reason_3', 'Reason_4']
            df.columns = column_names

            # re-order the columns in df
            column_names_reordered = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Date', 'Transportation Expense', 
                                      'Distance to Work', 'Daily Work Load Average', 'Body Mass Index', 'Education', 
                                      'Children', 'Pets', 'Age1', 'Age2', 'Age3', 'Age4', 'Absenteeism Time in Hours']
            df = df[column_names_reordered]
      
            # convert the 'Date' column into datetime
            df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

            # create a list with month values retrieved from the 'Date' column
            list_months = []
            for i in range(df.shape[0]):
                list_months.append(df['Date'][i].month)

            # insert the values in a new column in df, called 'Month Value'
            df['Month Value'] = list_months

            # create a new feature called 'Day of the Week'
            df['Day of the Week'] = df['Date'].apply(lambda x: x.weekday())


            # drop the 'Date' column from df
            df = df.drop(['Date'], axis = 1)

            # re-order the columns in df
            column_names_upd = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value', 'Day of the Week',
                                'Transportation Expense', 'Distance to Work',
                                'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children',
                                'Pets', 'Absenteeism Time in Hours', 'Age1', 'Age2', 'Age3', 'Age4']
            df = df[column_names_upd]


            # map 'Education' variables; the result is a dummy
            df['Education'] = df['Education'].map({1:0, 2:1, 3:1, 4:1})

            # replace the NaN values
            df = df.fillna(value=0)

            # drop the original absenteeism time
            df = df.drop(['Absenteeism Time in Hours'],axis=1)
            
            
            # we have included this line of checkpoint:
            self.data = df.copy()           
    
        # a function which outputs the probability of a data point to be 1
        def predicted_probability(self):
            if (self.data is not None):  
                pred = self.pipeline.predict_proba(self.data)[:,1]
                return pred
        
        # a function which outputs 0 or 1 based on our model
        def predicted_output_category(self):
            if (self.data is not None):
                pred_outputs = self.pipeline.predict(self.data)
                return pred_outputs
        
        # predict the outputs and the probabilities and 
        # add columns with these values at the end of the new data
        def predicted_outputs(self):
            if (self.data is not None):
                self.preprocessed_data['Probability'] = self.pipeline.predict_proba(self.data)[:,1]
                self.preprocessed_data ['Prediction'] = self.pipeline.predict(self.data)
                return self.preprocessed_data
